In [47]:
import pandas as pd
import numpy as np

In [54]:
#Selinle yaptığımız liste
all_list = []
for y in list(range(1995, 2021)):
    print(y, end='\r')
    raw = pd.read_csv(f'C:/Users/Berat/Documents/TEPAV/1)DATA/BACI/data/BACI_HS92_Y{y}_V202301.csv')
    country_list = pd.read_csv('C:/Users/Berat/Documents/TEPAV/1)DATA/BACI/data/country_codes_V202301.csv', encoding='latin-1')

    raw['q'] = [x.strip() for x in raw['q']]
    raw = raw[raw['q'] != 'NA']
    raw['q'] = [float(x) for x in raw['q']]

    hs4_list = [int(str(x)[:-2]) for x in raw['k']]
    raw['k_hs4'] = hs4_list

    hs4_list = [int(str(x)[:-4]) for x in raw['k']]
    raw['k_hs2'] = hs4_list

    country_list = country_list[['country_code', 'iso_3digit_alpha']]
    country_list.columns = ['i', 'ccode']
    raw = raw.merge(country_list, on = 'i', how = 'inner')
    country_list.columns = ['j', 'ccode']
    raw = raw.merge(country_list, on = 'j', how = 'inner')
    raw.columns = ['t', 'i', 'j', 'k', 'v', 'q', 'k_hs4', 'k_hs2', 'i_name', 'j_name']

    df_list = []
    c = 0
    for i in raw['i_name'].unique():
        df = raw[raw['i_name'] == i][['t', 'i_name', 'k_hs4', 'v']].groupby(['t', 'i_name', 'k_hs4']).sum().reset_index()
        df['i_share'] = df['v']/df['v'].sum()
        df_list.append(df)
        i_df = pd.concat(df_list)
        print('Exporter: ' + str(np.round(c/len(raw['i_name'].unique()),2)*100), end='\r')
        c = c + 1
    i_df.columns = ['t', 'i_name', 'k_hs4', 'i_v', 'i_share']

    df_list = []
    c = 0
    for j in raw['j_name'].unique():
        df = raw[raw['j_name'] == j][['t', 'j_name', 'k_hs4', 'v']].groupby(['t', 'j_name', 'k_hs4']).sum().reset_index()
        df['j_share'] = df['v']/df['v'].sum()
        df_list.append(df)
        j_df = pd.concat(df_list)
        print('Importer: ' + str(np.round(c/len(raw['j_name'].unique()),2)*100), end='\r')
        c = c + 1
    j_df.columns = ['t', 'j_name', 'k_hs4', 'j_v', 'j_share']

    df = i_df.merge(j_df, on = ['t', 'k_hs4'], how = 'outer')
    df = df[['t', 'i_name', 'j_name', 'k_hs4', 'i_v', 'j_v', 'i_share', 'j_share']].reset_index(drop = True)
    df['j_i_share_dif'] = abs(df['j_share'] - df['i_share'])

    df = df[['t', 'i_name', 'j_name', 'k_hs4', 'i_v', 'j_v', 'i_share', 'j_share', 'j_i_share_dif']].groupby(['t', 'i_name', 'j_name']).sum().reset_index()
    df['TCI'] = (1 - df['j_i_share_dif']/2)*100

    all_list.append(df)

In [3]:
raw = pd.read_csv(f'C:/Users/Berat/Documents/TEPAV/1)DATA/BACI/data/BACI_HS92_Y{2020}_V202201.csv')
country_list = pd.read_csv('C:/Users/Berat/Documents/TEPAV/1)DATA/BACI/data/country_codes_V202201.csv', encoding='latin-1')

raw['q'] = [x.strip() for x in raw['q']]
raw = raw[raw['q'] != 'NA']
raw['q'] = [float(x) for x in raw['q']]

hs4_list = [int(str(x)[:-2]) for x in raw['k']]
raw['k_hs4'] = hs4_list

hs4_list = [int(str(x)[:-4]) for x in raw['k']]
raw['k_hs2'] = hs4_list

country_list = country_list[['country_code', 'iso_3digit_alpha']]
country_list.columns = ['i', 'ccode']
raw = raw.merge(country_list, on = 'i', how = 'inner')
country_list.columns = ['j', 'ccode']
raw = raw.merge(country_list, on = 'j', how = 'inner')
raw.columns = ['t', 'i', 'j', 'k', 'v', 'q', 'k_hs4', 'k_hs2', 'i_name', 'j_name']

In [49]:
df_list = []
c = 0
for i in raw['i_name'].unique():
    df = raw[raw['i_name'] == i][['t', 'i_name', 'k_hs4', 'v']].groupby(['t', 'i_name', 'k_hs4']).sum().reset_index()
    df['i_share'] = df['v']/df['v'].sum()
    df_list.append(df)
    i_df = pd.concat(df_list)
    print('Exporter: ' + str(np.round(c/len(raw['i_name'].unique()),2)*100), end='\r')
    c = c + 1
i_df.columns = ['t', 'i_name', 'k_hs4', 'i_v', 'i_share']

df_list = []
c = 0
for j in raw['j_name'].unique():
    df = raw[raw['j_name'] == j][['t', 'j_name', 'k_hs4', 'v']].groupby(['t', 'j_name', 'k_hs4']).sum().reset_index()
    df['j_share'] = df['v']/df['v'].sum()
    df_list.append(df)
    j_df = pd.concat(df_list)
    print('Importer: ' + str(np.round(c/len(raw['j_name'].unique()),2)*100), end='\r')
    c = c + 1
j_df.columns = ['t', 'j_name', 'k_hs4', 'j_v', 'j_share']

In [50]:
df = i_df.merge(j_df, on = ['t', 'k_hs4'], how = 'outer')
df = df[['t', 'i_name', 'j_name', 'k_hs4', 'i_v', 'j_v', 'i_share', 'j_share']].reset_index(drop = True)
df['j_i_share_dif'] = abs(df['j_share'] - df['i_share'])

df = df[['t', 'i_name', 'j_name', 'k_hs4', 'i_v', 'j_v', 'i_share', 'j_share', 'j_i_share_dif']].groupby(['t', 'i_name', 'j_name']).sum().reset_index()
df['TCI'] = (1 - df['j_i_share_dif']/2)*100

In [51]:
df

,t,i_name,j_name,k_hs4,i_v,j_v,i_share,j_share,j_i_share_dif,TCI
0,2020,ABW,ABW,1853661,109670.732,5.547097e+05,0.944823,0.711585,1.292793,35.360355
1,2020,ABW,AFG,1888921,42666.894,3.422776e+06,0.367579,0.515125,0.584649,70.767566
2,2020,ABW,AGO,1897725,113173.578,4.323502e+06,0.975000,0.601923,1.350313,32.484326
3,2020,ABW,AIA,1511154,35391.717,4.885103e+04,0.304903,0.358489,0.522882,73.855909
4,2020,ABW,ALB,1905634,40937.553,3.301657e+06,0.352681,0.613353,0.719728,64.013608
...,...,...,...,...,...,...,...,...,...,...
51071,2020,ZWE,WSM,3154208,739698.137,2.764012e+05,0.171109,0.886504,0.958025,52.098773
51072,2020,ZWE,YEM,3656972,1176468.539,9.088897e+06,0.272143,0.870156,0.988308,50.584599
51073,2020,ZWE,ZAF,3915173,4322974.546,5.711520e+07,1.000000,0.811610,1.637695,18.115267
51074,2020,ZWE,ZMB,3870923,4171405.107,5.021698e+06,0.964939,0.871429,1.717886,14.105700
